In [2]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path
from PIL import Image

def find_matching_image(subfolder, target_year, tolerance=2):
    """
    Find high-resolution image matching the target year within tolerance
    
    Args:
        subfolder: Path to subfolder
        target_year: Target year (e.g., 2000, 2005)
        tolerance: Year tolerance (default ±2 years)
    
    Returns:
        Path to matching PNG image or None
    """
    year_pattern = r'(199[0-9]|20[0-1][0-9]|202[0-5])'
    
    for file in os.listdir(subfolder):
        if file.lower().endswith('.png'):
            match = re.search(year_pattern, file)
            if match:
                img_year = int(match.group(1))
                if abs(img_year - target_year) <= tolerance:
                    return os.path.join(subfolder, file), img_year
    
    return None, None

def extract_coordinates_from_meta(subfolder):
    """
    Extract coordinates from meta.txt file in the subfolder
    
    Args:
        subfolder: Path to subfolder containing meta.txt
    
    Returns:
        Coordinate string or None if not found
    """
    meta_path = os.path.join(subfolder, 'meta.txt')
    
    if not os.path.exists(meta_path):
        return None
    
    try:
        with open(meta_path, 'r', encoding='utf-8') as f:
            for line in f:
                # Look for the line with pixel bounds
                if '0.01deg pixel bounds' in line:
                    # Extract coordinates using regex
                    # Pattern: (L,B,R,T): -85.24, 30.6, -85.23, 30.61
                    match = re.search(r'\(L,B,R,T\):\s*([-\d.]+),\s*([-\d.]+),\s*([-\d.]+),\s*([-\d.]+)', line)
                    if match:
                        left, bottom, right, top = match.groups()
                        # Format as coordinate string
                        return f"({left}°, {bottom}°) to ({right}°, {top}°)"
    except Exception as e:
        print(f"Warning: Could not read meta.txt in {subfolder}: {e}")
    
    return None

def read_tif_bands(tif_path):
    """
    Read all bands from a multi-band TIF file
    
    Returns:
        numpy array with shape (bands, height, width)
    """
    with rasterio.open(tif_path) as src:
        # Read all bands
        data = src.read()  # Shape: (bands, height, width)
    return data

def create_comparison_figure(subfolder):
    """
    Create 3x5 comparison figure for a subfolder
    
    Rows:
    - Row 1: Forest area (green=forest, white=non-forest)
    - Row 2: Forest edge (heatmap of edge length)
    - Row 3: High-resolution images (stretched to same size)
    
    Columns: 2000, 2005, 2010, 2015, 2020
    """
    years = [2000, 2005, 2010, 2015, 2020]
    year_indices = {2000: 0, 2005: 1, 2010: 2, 2015: 3, 2020: 4}
    
    # Read TIF files
    area_tif = os.path.join(subfolder, 'area_5yr.tif')
    edge_tif = os.path.join(subfolder, 'edge_5yr.tif')
    
    if not os.path.exists(area_tif) or not os.path.exists(edge_tif):
        print(f"Skipping {subfolder}: Missing TIF files")
        return None
    
    area_data = read_tif_bands(area_tif)  # Shape: (5, height, width)
    edge_data = read_tif_bands(edge_tif)  # Shape: (5, height, width)
    
    # Get coordinates from meta.txt
    coordinates = extract_coordinates_from_meta(subfolder)
    if coordinates:
        title = f'Validation Sample: {coordinates}'
    else:
        # Fallback to folder name if meta.txt not found
        title = f'Validation Sample: {os.path.basename(subfolder)}'
    
    # Create figure with GridSpec for better control
    fig = plt.figure(figsize=(20, 12))
    import matplotlib.gridspec as gridspec
    
    # Create grid: 3 rows for data, 1 row for colorbar
    # Smaller colorbar row and tighter spacing
    gs = gridspec.GridSpec(4, 5, figure=fig, height_ratios=[1, 1, 0.08, 1], 
                          hspace=0.25, wspace=0.15)
    
    fig.suptitle(title, fontsize=16, fontweight='bold', y=0.98)
    
    # Create axes for each subplot
    axes = []
    for row in [0, 1, 3]:  # Skip row 2 (for colorbar)
        row_axes = []
        for col in range(5):
            if row == 3:
                gs_idx = gs[row, col]
            else:
                gs_idx = gs[row, col]
            ax = fig.add_subplot(gs_idx)
            row_axes.append(ax)
        axes.append(row_axes)
    
    # Process each year
    for col, year in enumerate(years):
        band_idx = year_indices[year]
        
        # Row 1: Forest Area
        area_band = area_data[band_idx]
        # Create binary mask: forest (>0) = dark green, non-forest = white
        forest_mask = np.zeros((*area_band.shape, 3))
        forest_mask[area_band > 0] = [0, 0.5, 0]  # Dark green
        forest_mask[area_band == 0] = [1, 1, 1]   # White
        
        axes[0][col].imshow(forest_mask, aspect='auto')
        axes[0][col].set_title(f'Forest Extent\n{year}', fontsize=10)
        axes[0][col].axis('off')
        
        # Row 2: Forest Edge
        edge_band = edge_data[band_idx]
        # Use colormap for edge length
        im = axes[1][col].imshow(edge_band, cmap='YlOrRd', vmin=0, aspect='auto')
        axes[1][col].set_title(f'Forest Edge\n{year}', fontsize=10)
        axes[1][col].axis('off')
        
        # Row 3: High-resolution image
        img_path, img_year = find_matching_image(subfolder, year, tolerance=2)
        
        if img_path:
            # Read and display image
            img = Image.open(img_path)
            img_array = np.array(img)
            # Use same aspect as other rows to maintain consistent subplot sizes
            axes[2][col].imshow(img_array, aspect='auto')
            axes[2][col].set_title(f'High-Res Image\n{img_year}',fontsize=10)
        else:
            # No matching image found
            axes[2][col].text(0.5, 0.5, 'No Image', 
                            ha='center', va='center',
                            transform=axes[2][col].transAxes,
                            fontsize=12, color='red')
            axes[2][col].set_title(f'High-Res Image\n{year} (N/A)', fontsize=10)
        
        axes[2][col].axis('off')
    
    # Add row labels on the left - unified style
    axes[0][0].set_ylabel('Forest Extent', fontsize=11, fontweight='bold', labelpad=10)
    axes[1][0].set_ylabel('Forest Edge Length', fontsize=11, fontweight='bold', labelpad=10)
    axes[2][0].set_ylabel('High-Resolution Image', fontsize=11, fontweight='bold', labelpad=10)
    
    # Add colorbar in the dedicated row between edge and images - centered
    # Use only 2 columns centered for a compact colorbar
    # Adjust position to move colorbar up within its row
    cbar_ax = fig.add_axes([0.375, 0.39, 0.25, 0.015])  # [left, bottom, width, height]
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Edge Length (m)', fontsize=9)
    cbar.ax.tick_params(labelsize=8)
    
    # Save figure
    output_path = os.path.join(subfolder, 'forest_change_comparison.png')
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Created: {output_path}")
    return output_path

def process_all_subfolders(root_folder):
    """
    Process all subfolders and create comparison figures
    """
    output_files = []
    
    # Find all subfolders
    for root, dirs, files in os.walk(root_folder):
        # Check if this folder contains the required TIF files
        if 'area_5yr.tif' in files and 'edge_5yr.tif' in files:
            print(f"\nProcessing: {root}")
            output = create_comparison_figure(root)
            if output:
                output_files.append(output)
    
    return output_files

if __name__ == "__main__":
    # Your folder path
    root_folder = r"G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output"
    
    print("=" * 80)
    print("Forest Change Comparison Figure Generator")
    print("=" * 80)
    print(f"Root folder: {root_folder}")
    print(f"Target years: 2000, 2005, 2010, 2015, 2020")
    print(f"Image matching tolerance: ±2 years")
    print("=" * 80)
    
    if not os.path.exists(root_folder):
        print(f"Error: Folder does not exist: {root_folder}")
    else:
        output_files = process_all_subfolders(root_folder)
        
        print("\n" + "=" * 80)
        print(f"Complete! Generated {len(output_files)} comparison figures")
        print("=" * 80)
        
        if output_files:
            print("\nGenerated files:")
            for i, file in enumerate(output_files[:10], 1):
                print(f"  {i}. {file}")
            if len(output_files) > 10:
                print(f"  ... and {len(output_files) - 10} more files")

Forest Change Comparison Figure Generator
Root folder: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output
Target years: 2000, 2005, 2010, 2015, 2020
Image matching tolerance: ±2 years

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00106
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00106\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00113
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00113\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00114
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_

✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00540\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00541
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00541\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00542
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00542\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00545
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_00545\forest_change_comp

✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01030_B\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01038_B
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01038_B\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01040_A
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01040_A\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01044_C
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01044_C\fore

✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01549\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01561
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01561\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01571
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01571\forest_change_comparison.png

Processing: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01581
✓ Created: G:\Hangkai\Global_Forest_edge_mapping_data\validation_samples_0p01deg_per_sample_folder\samples\output\sample_01581\forest_change_comp